# Ramsey King
# DSC 630 - Predictive Analytics
# October 16, 2021
# Exercise 7.2

In [2]:
import pandas as pd
import numpy as np

hotel_destination_df = pd.read_csv('files/destinations.csv')
train_df = pd.read_csv('files/train.csv')

In [3]:
# Explore train.csv
print(train_df.shape)
train_df.head(7)

(37670293, 24)


,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21
5,2014-08-09 18:13:12,2,3,66,442,35390,911.5142,93,0,0,...,0,1,14984,1,0,1,2,50,1457,92
6,2014-07-16 09:42:23,2,3,66,189,10067,NaN,501,0,0,...,0,1,8267,1,0,2,2,50,675,41


Wow!  37 million rows of data will probably take forever to do any analysis for a homework assignment.  After doing some exploratory data analysis, I will limit the data set.  I will first limit it 1 million rows, and then see what happens from there.

In [4]:
print("Describe Data")
print(train_df.describe())

print("Summarized Data")
train_df.describe(include=['O'])

Describe Data
          site_name  posa_continent  user_location_country  \
count  3.767029e+07    3.767029e+07           3.767029e+07   
mean   9.795271e+00    2.680473e+00           8.610880e+01   
std    1.196754e+01    7.480393e-01           5.924310e+01   
min    2.000000e+00    0.000000e+00           0.000000e+00   
25%    2.000000e+00    3.000000e+00           6.600000e+01   
50%    2.000000e+00    3.000000e+00           6.600000e+01   
75%    1.400000e+01    3.000000e+00           7.000000e+01   
max    5.300000e+01    4.000000e+00           2.390000e+02   

       user_location_region  user_location_city  orig_destination_distance  \
count          3.767029e+07        3.767029e+07               2.414529e+07   
mean           3.084060e+02        2.775304e+04               1.970090e+03   
std            2.084437e+02        1.678255e+04               2.232442e+03   
min            0.000000e+00        0.000000e+00               5.600000e-03   
25%            1.740000e+02        1.

,date_time,srch_ci,srch_co
count,37670293,37623210,37623209
unique,25419546,1269,1262
top,2014-11-06 17:34:48,2014-12-26,2015-01-02
freq,15,223153,218447


In [5]:
# want to take a look at the column names
train_df.columns

Index(['date_time', 'site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster'],
      dtype='object')

In [6]:
#unique values in the columns
'''for col in train_df:
    print(train_df[col].unique())'''

'for col in train_df:\n    print(train_df[col].unique())'

In [7]:
# going to randomly select 500,000 rows
hotel_cluster_df = train_df.sample(n=500000)
print(hotel_cluster_df['hotel_cluster'])
hotel_cluster_df.shape

36732766    89
20277215     2
23404975    91
20969207    10
2144632     30
            ..
3342677     82
23452602    83
4844021     48
4700696     89
13523751    40
Name: hotel_cluster, Length: 500000, dtype: int64


(500000, 24)

In [8]:
# create a column that will calculate the length of stay in days
hotel_cluster_df['length_of_stay'] = pd.to_datetime(hotel_cluster_df['srch_co']) - pd.to_datetime(hotel_cluster_df['srch_ci'])
hotel_cluster_df['length_of_stay'] = hotel_cluster_df['length_of_stay'].apply(lambda x: x.days)
hotel_cluster_df['length_of_stay'].fillna(1000, inplace=True)
hotel_cluster_df['length_of_stay'][:7]

36732766    2.0
20277215    5.0
23404975    3.0
20969207    7.0
2144632     2.0
16674193    1.0
13766342    5.0
Name: length_of_stay, dtype: float64

In [9]:
# Removing the time columns (date_time, srch_ci, srch_co) and then removing the target column hotel_cluster

# columns_to_drop = ['date_time', 'srch_ci', 'srch_co']
# hotel_cluster_df.drop(columns=columns_to_drop, inplace=True)
features = ['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt',
       'hotel_continent', 'hotel_country', 'hotel_market', 'length_of_stay'
       ]
x = hotel_cluster_df.loc[:, features].values
y = hotel_cluster_df.loc[:, ['hotel_cluster']].values

In [10]:
# merge the datasets from the hotel_cluster_df and hotel_destination_df together
print(hotel_cluster_df.head())
hotel_destination_df.head()

expedia_df = pd.merge(hotel_cluster_df, hotel_destination_df, left_on='srch_destination_id', right_index=True)
expedia_df.head()

                    date_time  site_name  posa_continent  \
36732766  2014-06-07 02:25:12         24               2   
20277215  2013-04-13 04:24:29          2               3   
23404975  2014-12-08 13:55:40          2               3   
20969207  2013-05-11 19:18:16         37               1   
2144632   2014-06-30 15:10:19         11               3   

          user_location_country  user_location_region  user_location_city  \
36732766                      3                    50                5703   
20277215                     66                   314                4868   
23404975                     66                   184                2037   
20969207                     69                   747                6783   
2144632                     205                   155               40095   

          orig_destination_distance  user_id  is_mobile  is_package  ...  \
36732766                        NaN   242642          0           1  ...   
20277215                

,srch_destination_id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
36732766,635,2014-06-07 02:25:12,24,2,3,50,5703,NaN,242642,0,...,-2.189452,-2.235919,-2.234932,-2.235919,-2.235919,-2.232275,-2.235919,-2.235919,-2.235919,-2.205414
15354788,635,2014-08-17 21:01:08,24,2,3,51,41641,NaN,1197368,0,...,-2.189452,-2.235919,-2.234932,-2.235919,-2.235919,-2.232275,-2.235919,-2.235919,-2.235919,-2.205414
27387853,635,2013-07-30 12:32:08,33,2,194,38,42328,NaN,946438,0,...,-2.189452,-2.235919,-2.234932,-2.235919,-2.235919,-2.232275,-2.235919,-2.235919,-2.235919,-2.205414
30036866,635,2014-03-08 23:34:05,24,2,3,64,3169,NaN,713334,0,...,-2.189452,-2.235919,-2.234932,-2.235919,-2.235919,-2.232275,-2.235919,-2.235919,-2.235919,-2.205414
30791484,635,2014-10-25 00:25:15,24,2,3,50,5224,NaN,242477,0,...,-2.189452,-2.235919,-2.234932,-2.235919,-2.235919,-2.232275,-2.235919,-2.235919,-2.235919,-2.205414


In [28]:
expedia_df.columns

Index(['srch_destination_id', 'date_time', 'site_name', 'posa_continent',
       'user_location_country', 'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile',
       ...
       'd140', 'd141', 'd142', 'd143', 'd144', 'd145', 'd146', 'd147', 'd148',
       'd149'],
      dtype='object', length=176)

In [26]:
print(expedia_df['orig_destination_distance'].count())
# expedia_df['orig_destination_distance'].fillna(-1, inplace=True)
expedia_df['orig_destination_distance'].isnull().sum()


499008


0

In [32]:
# separate features from target
features = ['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt',
       'hotel_continent', 'hotel_country', 'hotel_market', 'length_of_stay'
       ]
x = expedia_df.loc[:, features].values
y = expedia_df.loc[:, ['hotel_cluster']].values

In [34]:
# standardize the data set
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
expedia_scaled_df = scale.fit_transform(x)
expedia_scaled_df

array([[ 1.18436782, -0.90701955, -1.40106421, ...,  1.44017475,
         0.85879835, -0.07432334],
       [ 1.18436782, -0.90701955, -1.40106421, ...,  1.44017475,
         0.85879835, -0.04451513],
       [ 1.9360639 , -0.90701955,  1.82171174, ...,  1.44017475,
         0.85879835, -0.07432334],
       ...,
       [-0.65311149,  0.4276762 , -0.33805434, ..., -0.5558619 ,
        -0.10908005, -0.01470692],
       [-0.65311149,  0.4276762 , -0.33805434, ..., -0.5558619 ,
         0.10600404, -0.04451513],
       [ 2.27015104, -2.2417153 , -0.28743482, ...,  2.25997552,
         1.89902322, -0.10413155]])

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

In [36]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
y_lr_train_pred = lr.predict(x_train)
y_lr_test_pred = lr.predict(x_test)
lr.score(x_test, y_test)

0.006029909942670941

In [41]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=2, random_state=42)
rf.fit(x_train, y_train)
y_rf_train_pred = rf.predict(x_train)
y_rf_test_pred = rf.predict(x_test)
rf.score(x_test, y_test)

C:\Users\Ramsey\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.010065272203140796

In [12]:
# will normalize the destinations.csv file to create some uniformity.

'''from sklearn import preprocessing
# hotel_destination_df.set_index('srch_destination_id', inplace=True)
scaler = preprocessing.MinMaxScaler()
names = hotel_destination_df.columns
new_vals = scaler.fit_transform(hotel_destination_df)
hotel_destination_scaled_df = pd.DataFrame(new_vals, columns=names)
print(hotel_destination_scaled_df.head())
clusters = hotel_cluster_df.hotel_cluster.unique()
print(sorted(clusters), len(clusters))
hotel_destination_scaled_df.shape'''


"from sklearn import preprocessing\n# hotel_destination_df.set_index('srch_destination_id', inplace=True)\nscaler = preprocessing.MinMaxScaler()\nnames = hotel_destination_df.columns\nnew_vals = scaler.fit_transform(hotel_destination_df)\nhotel_destination_scaled_df = pd.DataFrame(new_vals, columns=names)\nprint(hotel_destination_scaled_df.head())\nclusters = hotel_cluster_df.hotel_cluster.unique()\nprint(sorted(clusters), len(clusters))\nhotel_destination_scaled_df.shape"

In [13]:
# PCA
'''from sklearn.decomposition import PCA

principal = PCA(n_components=3)
principal.fit(hotel_destination_scaled_df)
x = principal.transform(hotel_destination_scaled_df)
x.shape'''

'from sklearn.decomposition import PCA\n\nprincipal = PCA(n_components=3)\nprincipal.fit(hotel_destination_scaled_df)\nx = principal.transform(hotel_destination_scaled_df)\nx.shape'

In [14]:
#
# print(principal.explained_variance_ratio_, sum(principal.explained_variance_ratio_))

In [15]:
# print(pd.DataFrame(principal.components_,columns=hotel_destination_scaled_df.columns,index = ['PC-1','PC-2','PC-3']))
